# Team EGE - BLG453E Term Project
Egehan Orta - 150160124

Yavuz Ege Okumuş - 150160118
## Vascular Segmentation

In [1]:
import numpy as np 
import cv2 as cv
import nibabel as nib
import matplotlib.pyplot as plt
from itertools import product

In [2]:
img_dict = {
    'img' : nib.load("V.nii").get_fdata(),
    'seg' : nib.load("V_seg.nii").get_fdata(),
}

In [3]:
seed_2d = [[82, 36, 43],[35, 64, 48],[156, 63, 50],[82, 34, 44],
           [45, 58, 46],[167, 76, 49],[71, 19, 53],[91, 3, 70],
           [33, 61, 51],[78, 37, 54],[83, 34, 55],[115, 3, 80],
           [31, 68, 47],[29, 19, 45],[48, 41, 52]]

seed_3d = [[82, 21, 47],[46, 45, 51],[35, 31, 52],
           [91, 22, 46],[2, 23, 48],[35, 31, 50]]

In [4]:
def region_growing_2D(img, s, thresh, mode):
    x,y,z = img.shape
    
    p = []
    p.append(s)
    ret = np.zeros_like(img, dtype=np.float32)
    ret[s[0], s[1], s[2]] = 1.0
    
    n8 = [[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1]]
    n4 = [[-1, 0], [0, -1], [0, 1], [1, 0]]
    
    if mode == 8:
        sel = n8
    elif mode == 4:
        sel = n4
        
    while p:
        i = p.pop(0)
        for item in sel:
            temp = [i[0]+item[0], i[1]+item[1], s[2]]
            if (temp[0]<0) or (temp[0]>x-1) or (temp[1]<0)\
            or (temp[1]>y-1) or (temp[2]<0) or (temp[2]>z-1)\
            or (ret[temp[0], temp[1], temp[2]] == 1.0):
                continue
            if img[temp[0], temp[1], temp[2]] > thresh:
                ret[temp[0], temp[1], temp[2]] = 1.0
                p.append(temp)
    return ret

In [5]:
def region_growing_3D(img, s, thresh, mode):
    x,y,z = img.shape
    
    p = []
    p.append(s)
    ret = np.zeros_like(img, dtype=np.float32)
    ret[s[0], s[1], s[2]] = 1.0
    
    n26 = [[-1, -1, -1],[-1, -1, 0],[-1, -1, 1],[-1, 0, -1],[-1, 0, 0],
           [-1, 0, 1],[-1, 1, -1],[-1, 1, 0],[-1, 1, 1],[0, -1, -1],
           [0, -1, 0],[0, -1, 1],[0, 0, -1],[0, 0, 1],[0, 1, -1],
           [0, 1, 0],[0, 1, 1],[1, -1, -1],[1, -1, 0],[1, -1, 1],
           [1, 0, -1],[1, 0, 0],[1, 0, 1],[1, 1, -1],[1, 1, 0],[1, 1, 1]]
    
    n6 = [[-1,0,0], [1,0,0], [0,-1,0], [0,1,0],[0,0,-1],[0,0,1]]
    
    if mode == 26:
        sel = n26
    elif mode==6:
        sel = n6
    while p:
        i = p.pop(0)
        
        for item in sel:
            temp = [i[0]+item[0], i[1]+item[1], i[2]+item[2]]
            if (temp[0]<0) or (temp[0]>x-1) or (temp[1]<0)\
            or (temp[1]>y-1) or (temp[2]<0) or (temp[2]>z-1)\
            or (ret[temp[0], temp[1], temp[2]] == 1.0):
                continue
            if img[temp[0], temp[1], temp[2]] > thresh:
                ret[temp[0], temp[1], temp[2]] = 1.0
                p.append(temp)
    return ret

In [6]:
def dice_score(gt,seg):
    return 2.0 * np.count_nonzero( np.logical_and(gt,seg)) / (np.count_nonzero(gt)+ np.count_nonzero(seg))

In [8]:
imlist=[]
for s in seed_2d:
    imlist.append(region_growing_2D(img_dict['img'],s,0.65,8))
mask = np.zeros_like(img_dict['img'])
for i in imlist:
    mask = np.logical_or(i,mask)
    
img_dict['img8'] = mask

In [10]:
imlist=[]
for s in seed_2d:
    imlist.append(region_growing_2D(img_dict['img'],s,0.65,4))
mask = np.zeros_like(img_dict['img'])
for i in imlist:
    mask = np.logical_or(i,mask)
    
img_dict['img4'] = mask

In [12]:
imlist=[]
for s in seed_3d:
    imlist.append(region_growing_3D(img_dict['img'],s,0.7,26))
mask = np.zeros_like(img_dict['img'])
for i in imlist:
    mask = np.logical_or(i,mask)
    
img_dict['img26'] = mask

In [14]:
imlist=[]
for s in seed_3d:
    imlist.append(region_growing_3D(img_dict['img'],s,0.7,6))
mask = np.zeros_like(img_dict['img'])
for i in imlist:
    mask = np.logical_or(i,mask)
    
img_dict['img6'] = mask

In [16]:
scores = {
    'img8' : dice_score(img_dict['seg'], img_dict['img8']),
    'img4' : dice_score(img_dict['seg'], img_dict['img4']),
    'img26' : dice_score(img_dict['seg'], img_dict['img26']),
    'img6' : dice_score(img_dict['seg'], img_dict['img6'])
}

In [28]:
print('Scores')
print('='*30)
for key in scores.keys():
    print(key,': ',scores[key] ,sep='')

Scores
img8: 0.9070557491289198
img4: 0.9062309102015883
img26: 0.9074053188369837
img6: 0.9083026621807754
